In [2]:
import numpy as np
import pandas as pd


import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data
from torch_geometric.nn import GraphConv, global_add_pool
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_networkx, from_networkx


import networkx as nx

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings("ignore")


torch.manual_seed(1)
torch.use_deterministic_algorithms(False)

AttributeError: module 'typing' has no attribute '_ClassVar'

In [2]:
torch.__version__

'1.12.0+cu113'

# dataset preparations 

## custom dataset

In [24]:
class CustomMDGATData(Dataset):
    r"""creating the pytorch dataset from the saved sql database
    
    Args:
        db_dir : dir to database with different tables representing the propeties
        
    """
    def __init__(self, db_dir):
        try:
            conn = sqlite3.connect(db_dir)
        except Error:
            print(Error)
        
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table'; ")
        self._table_names = cursor.fetchall()
        
        self._dataframes = {}
        
        for table in self._table_names:
            self._dataframes[table[0]] = pd.read_sql(f"SELECT * FROM {table[0]}", conn)
        
    def _choose_df(self, df_name):
        df = self._dataframes[df_name].drop(['index'], axis=1)
        df.columns = df.columns.astype(int)
        return df
    
    def __len__(self):
        return len(self._choose_df('trajectory'))
        
    def __getitem__(self, idx):
        trajectory = torch.tensor(self._choose_df('trajectory')\
                                  .loc[idx].to_numpy().reshape(-1, 3), dtype=torch.float32)
        force = torch.tensor(self._choose_df('force')\
                             .loc[idx].to_numpy().reshape(-1, 3), dtype=torch.float32)
        energy = torch.tensor(self._choose_df('energies')\
                              .loc[idx].to_numpy(), dtype=torch.float32)
        elements = self._choose_df('elements').loc[idx].to_list()
        
        z = self._choose_df('z').loc[idx].to_list()
        
        return {'trajectory' : trajectory, 'force' : force, 
                'energy' : energy, 'elements' : elements, 'z' : z}

In [175]:
peptide_dataset = torch.load('')

RuntimeError: [enforce fail at inline_container.cc:110] . file in archive is not in a subdirectory: R.npy

In [15]:
train_data, test_data = torch.utils.data.random_split(
    peptide_graphData, 
    [3000, 1108], 
    generator=torch.Generator().manual_seed(1)
)

NameError: name 'peptide_graphData' is not defined

In [21]:
train_loader = geom_data.DataLoader(train_data, batch_size, shuffle=True)

test_loader = geom_data.DataLoader(test_data, batch_size)
batch = next(iter(train_loader))

In [4]:
peptide_dataset[0]

{'trajectory': tensor([[20.6365,  7.4573, 10.2589],
         [20.8772,  8.0381,  8.9844],
         [19.5590,  8.2561,  8.2748],
         [19.4328,  9.1515,  7.4074],
         [21.4657,  8.9951,  8.9941],
         [21.4325,  7.3725,  8.3204],
         [20.4970,  8.2285, 10.8920],
         [21.3709,  6.8386, 10.5706],
         [18.6094,  7.4091,  8.6391],
         [17.2701,  7.4081,  8.0193],
         [17.1773,  7.0648,  6.5617],
         [16.1413,  7.1310,  5.9130],
         [16.6815,  6.6571,  8.5105],
         [16.7327,  8.2857,  8.2390],
         [18.8936,  6.8862,  9.4666],
         [18.2978,  6.6908,  5.9187],
         [18.3800,  6.5380,  4.5134],
         [18.1985,  7.8844,  3.7685],
         [17.6035,  7.9065,  2.7031],
         [19.4454,  6.1644,  4.3119],
         [17.6246,  5.7783,  4.1568],
         [19.1399,  6.6516,  6.4866],
         [18.8090,  8.9710,  4.3226],
         [18.7881, 10.3292,  3.6948],
         [17.4576, 11.0241,  3.7655],
         [17.3465, 12.0666,  4.4646]

In [65]:
train_data[0]

NameError: name 'train_data' is not defined

## QM9 dataset

In [4]:
from torch_geometric.datasets import QM9

In [5]:
qm9_dataset = QM9(root='/p/home/jusers/kotobi2/juwels')

In [6]:
#train and test data
qm9_dataset.shuffle()

train_dataset = qm9_dataset[:20000]
test_dataset = qm9_dataset[20001:24000]

qm9_train_loader = geom_data.DataLoader(train_dataset, batch_size=64, shuffle=True)
qm9_test_loader = geom_data.DataLoader(test_dataset, batch_size=64)

In [7]:
batch = next(iter(qm9_train_loader))
batch#.y[:, 10].unsqueeze(1)#.shape

DataBatch(x=[1048, 11], edge_index=[2, 2122], edge_attr=[2122, 4], y=[64, 19], pos=[1048, 3], idx=[64], name=[64], z=[1048], batch=[1048], ptr=[65])

# Utils 

## RBF 

In [2]:
class RBFExpansion(nn.Module):
    """class to expand the distances between nodes (atoms) by RBF"""
    def __init__(self, r_max=1, r_min=0, gap=0.1):
        super(RBFExpansion, self).__init__()
        num_centers = int(np.ceil((r_max - r_min) / gap))
        self.centers = np.linspace(r_max, r_min, num_centers)
        self.centers = nn.Parameter(torch.tensor(self.centers).float(), requires_grad=False)
        self.gamma = 1 / gap
        
        
    def reset_parameters(self):
        device = self.centers.device
        self.centers = nn.Parameter(self.centers.clone().detach().float(), requires_grad=False)
        
    def _minmax_rij(self, r_ij):
        return MinMaxScaler().fit_transform(r_ij)
    
    def _standardScaler(self, r_ij):
        return StandardScaler().fit_transform(r_ij)
        
    def forward(self, r_ij, minmax=False, standardScaler=False):
        if minmax:
            r_ij = torch.tensor(self._minmax_rij(r_ij.reshape(-1, 1)))
        elif standardScaler:
            r_ij = torch.tensor(self._standardScaler(r_ij).reshape(-1, 1))
        else:
            r_ij = r_ij.reshape(-1, 1)
        radial = r_ij - self.centers
        coef = - self.gamma
        return torch.exp(coef * (radial ** 2))

In [76]:
class RBF(nn.Module):
    def __init__(self, out_features, kernel):
        super(RBF, self).__init__()
        self.out_features = out_features
        self.centers = nn.Parameter(torch.Tensor(out_features))
        self.log_sigmas = nn.Parameter(torch.Tensor(out_features))
        self.kernel = kernel
        self.reset_parameters()
        
    def reset_parameters(self):
        nn.init.normal_(self.centers, 0, 1)
        nn.init.constant_(self.log_sigmas, 0)
                
    
    def forward(self, inputs, resize_shape:int):
        size = (-1, self.out_features)
        x = inputs.expand(size)
        c = self.centers.unsqueeze(0).expand(size)
        print(x.size(), c.size())
        distances = (x-c).pow(2).sum(-1).pow(0.5) #/ torch.exp(self.log_sigmas).unsqueeze(0)
        rbf_distances = self.kernel(distances)
        return rbf_distances.view(resize_shape, -1)
    
#defining kernels 

def gaussian(alpha):
    return torch.exp(-1*alpha.pow(2))

def linear(alpha):
    return alpha

def quadratic(alpha):
    return alpha.pow(2)

def multiquadratic(alpha):
    return (torch.ones_like(alpha) + alpha.pow(2)).pow(0.5)

def inverse_multiquadratic(alpha):
    return torch.ones_like(alpha)/(torch.ones_like(alpha) + alpha.pow(2)).pow(0.5)

# Model architecture 

## Using sum of interaction layers 

In [102]:
class InitialEmbedding(nn.Module):
    def __init__(self, emb_size, init_method='kaiming'):
        super().__init__()
        self.embedding = nn.Embedding(95, emb_size)
        self.init_method = init_method
        
    def _init_parameters(self):
            for layer in [*self.fc, self.embedding]:
                if self.init_method == 'xavier':
                    if isinstance(layer, nn.Linear):
                        nn.init.xavier_uniform_(layer.weight)
                elif self.init_method == 'kaiming':
                    if isinstance(layer, nn.Linear):
                        nn.init.kaiming_uniform_(layer.weight, mode='fan_in')
                else:
                    raise NotImplementedError
        
    def forward(self, z):
        z = self.embedding(z)
        return z

In [39]:
class Interaction_layer(nn.Module):
    def __init__(self, c_in, c_hidd, c_out, in_heads=1, out_heads=1):
        super().__init__()
        self.gat = geom_nn.GATv2Conv(
            c_in,
            c_hidd,
            heads=in_heads,
        )
        self.relu = nn.ReLU()
        self.gat2 = geom_nn.GATv2Conv(
            c_hidd*in_heads,
            c_out,
            heads=out_heads,
        )
        
    def forward(self, x, edge_index):
        x = self.gat(x, edge_index)
        x = self.relu(x)
        x = self.gat2(x, edge_index)
        #x = geom_nn.global_add_pool(x, batch_idx)
        return x

In [40]:
class Output_block(nn.Module):
    def __init__(self, c_in, c_hidd, c_out, num_linears=6):
        super().__init__()
        output_layers = []
        for i in range(num_linears):
            output_layers += [nn.Linear(c_in, c_hidd),
                             nn.ReLU()]
            c_in = c_hidd
            
        output_layers += [nn.Linear(c_hidd, c_out)]
        self.output_layers = nn.ModuleList(output_layers)
        
    def _init_parameters(self):
        for layer in self.output_layers:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight)
        
    def forward(self, x):
        for layer in self.output_layers:
            x = layer(x)
        return x

In [16]:
class Residual_layer(nn.Module):
    def __init__(self,  

SyntaxError: unexpected EOF while parsing (3274572245.py, line 2)

In [14]:
test_interaction = Interaction_layer(4, 56, 30)
for batch_train in train_loader:
    batch, x, edge_index= (
         batch_train.batch, batch_train.atom_type, batch_train.edge_index)
    
    x = test_interaction(x.float(), edge_index, batch)
    print(x.size())  

NameError: name 'train_loader' is not defined

In [81]:
class GATModel(nn.Module):
    def __init__(self, num_interactions):
        super().__init__()
        self.num_layers = num_interactions
        
        self.embedding = InitialEmbedding(
            emb_size=3,
            init_method='kaiming',
        )
        
        self.rbf_layer = RBF(30, gaussian)
        
        self.out_blocks = []
        self.int_blocks = []
        for i in range(num_interactions):
            self.int_blocks += [Interaction_layer(44, 56, 30)]
            self.out_blocks += [Output_block(30, 100, 1, num_linears=3)]
        
        self.int_blocks = nn.ModuleList(self.int_blocks)
        self.out_blocks = nn.ModuleList(self.out_blocks) 
        
    def _calculate_Dij(self, batch):
        #finding the unique elements         
        batch_split = torch.cumsum(torch.unique(batch.batch, return_counts=True)[1], dim=0)[:-1]
               
        #splitting the atomic positions in the batch
        splitted_batch = torch.tensor_split(batch.pos, batch_split.cpu())
        
        all_Dij = []
        for atom_pos in splitted_batch:
            Dij = atom_pos[:, None, :] - atom_pos[None, :, :]
            Dij = torch.linalg.norm(Dij, axis=-1)
            Dij_max, _ = torch.max(Dij, dim=1, keepdim=True)
            Dij_min, _ = torch.min(Dij, dim=1, keepdim=True)
            Dij = (Dij - Dij_min) / (Dij_max - Dij_min)
            all_Dij.append(Dij.view(-1, 1))
        return torch.cat(all_Dij, dim=0)
        
        
    def forward(self, inputs):
        batch, x, edge_index, z, pos = (
         inputs.batch, inputs.x, inputs.edge_index, inputs.z, inputs.pos)
        
        Dij = self._calculate_Dij(inputs)
        
        rbf_Dij = self.rbf_layer(Dij, inputs.pos.size(0))
        
        z = self.embedding(z)
       
        x = torch.cat((x, rbf_Dij, z), 1) #r_ij.to('cuda:0'),
        
        x_hat = self.int_blocks[0](x.float(), edge_index)
        P = self.out_blocks[0](x_hat)
        
        for i in range(self.num_layers-1):
            x_hat = self.int_blocks[i+1](x.float(), edge_index)
            P += self.out_blocks[i+1](x_hat)
            
        P = geom_nn.global_add_pool(P, batch)
        
        return P 

In [82]:
learning_rate = 1e-4
batch_size = 100
epochs = 200

In [44]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [84]:
test_model = GATModel(num_interactions=4).to(device)
#test_model = nn.DataParallel(test_model)

In [16]:
test_model

GATModel(
  (embedding): InitialEmbedding(
    (embedding): Embedding(95, 3)
  )
  (rbf_layer): RBF()
  (int_blocks): ModuleList(
    (0): Interaction_layer(
      (gat): GATv2Conv(44, 56, heads=1)
      (relu): ReLU()
      (gat2): GATv2Conv(56, 30, heads=1)
    )
    (1): Interaction_layer(
      (gat): GATv2Conv(44, 56, heads=1)
      (relu): ReLU()
      (gat2): GATv2Conv(56, 30, heads=1)
    )
    (2): Interaction_layer(
      (gat): GATv2Conv(44, 56, heads=1)
      (relu): ReLU()
      (gat2): GATv2Conv(56, 30, heads=1)
    )
    (3): Interaction_layer(
      (gat): GATv2Conv(44, 56, heads=1)
      (relu): ReLU()
      (gat2): GATv2Conv(56, 30, heads=1)
    )
    (4): Interaction_layer(
      (gat): GATv2Conv(44, 56, heads=1)
      (relu): ReLU()
      (gat2): GATv2Conv(56, 30, heads=1)
    )
    (5): Interaction_layer(
      (gat): GATv2Conv(44, 56, heads=1)
      (relu): ReLU()
      (gat2): GATv2Conv(56, 30, heads=1)
    )
  )
  (out_blocks): ModuleList(
    (0): Output_block(

In [85]:
#loss_fn = nn.MSELoss()
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(test_model.parameters(), 1e-4)

In [86]:
for total_epochs in range(100):
    epoch_loss = 0
    total_graphs = 0
    test_model.train()
    for batch in qm9_train_loader:
        with torch.autograd.detect_anomaly():
            batch.to(device)
            optimizer.zero_grad()
            output = test_model(batch)
            loss = loss_fn(
                 output,batch.y[:, 10].unsqueeze(1)) #y[:, 10].unsqueeze(1)
            
            loss.backward()
            epoch_loss += loss.item()
            total_graphs += batch.num_graphs
            optimizer.step()
    train_avg_loss = epoch_loss / total_graphs
    val_loss = 0
    total_graphs = 0
    test_model.eval()
    for batch in qm9_test_loader:
        batch.to(device)
        output = test_model(batch)
        loss = loss_fn(
             output, batch.y[:, 10].unsqueeze(1))
        val_loss += loss.item()
        total_graphs += batch.num_graphs
    val_avg_loss = val_loss / total_graphs
    print(f"Epochs: {total_epochs} | "
           f"epoch avg. loss: {train_avg_loss:.2f} | "
           f"validation avg. loss: {val_avg_loss:.2f}")

torch.Size([1061, 11]) torch.Size([1061])
torch.Size([13, 13])
torch.Size([20, 20])
torch.Size([17, 17])
torch.Size([16, 16])
torch.Size([15, 15])
torch.Size([18, 18])
torch.Size([12, 12])
torch.Size([15, 15])
torch.Size([11, 11])
torch.Size([14, 14])
torch.Size([16, 16])
torch.Size([14, 14])
torch.Size([13, 13])
torch.Size([17, 17])
torch.Size([19, 19])
torch.Size([18, 18])
torch.Size([14, 14])
torch.Size([16, 16])
torch.Size([19, 19])
torch.Size([15, 15])
torch.Size([23, 23])
torch.Size([17, 17])
torch.Size([17, 17])
torch.Size([18, 18])
torch.Size([12, 12])
torch.Size([15, 15])
torch.Size([20, 20])
torch.Size([10, 10])
torch.Size([18, 18])
torch.Size([15, 15])
torch.Size([16, 16])
torch.Size([22, 22])
torch.Size([18, 18])
torch.Size([20, 20])
torch.Size([20, 20])
torch.Size([12, 12])
torch.Size([20, 20])
torch.Size([17, 17])
torch.Size([13, 13])
torch.Size([20, 20])
torch.Size([17, 17])
torch.Size([15, 15])
torch.Size([23, 23])
torch.Size([16, 16])
torch.Size([19, 19])
torch.Size([1

RuntimeError: shape '[1061, -1]' is invalid for input of size 18177

# Coding different types of message passing layers in pytorch

## learning the torch scatter

In [1]:
from torch_scatter import scatter_add
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

OSError: libcusparse.so.11: cannot open shared object file: No such file or directory

In [7]:
num_nodes = 4
embed_size = 5

src = torch.randint(0, num_nodes, (num_nodes, embed_size))
src_index = torch.tensor([0,0,0,1,1,2,3,3])
tmp = torch.index_select(src, 0, src_index)

In [11]:
target_index = torch.tensor([1,2,3,3,0,0,0,2])
aggr = scatter_add(tmp, target_index, 0)

In [25]:
index2 = target_index.expand((embed_size, target_index.size(0))).T
aggr2 = torch.zeros(num_nodes, embed_size, dtype=tmp.dtype).scatter_add(0, index2, tmp)

## Graph convolution layer

In [17]:
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

In [51]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')
        self.lin = nn.Linear(in_channels, out_channels, bias=False)
        self.bias = nn.Parameter(torch.Tensor(out_channels))
        
        self.reset_parameters()
        
    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()
        
    def forward(self, x, edge_index):
        edge_index = add_self_loops(edge_index, num_nodes=x.size(0))
        x = self.lin(x)
        
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row]*deg_inv_sqrt[col]
        
        out = self.propagate(edge_index, x=x, norm=norm)
        
        out += self.bias
        
        return out
    
    def message(self, x_j, norm):
        
        return norm.view(-1, 1) * x_j

In [55]:
gcn = GCNConv(11, 30)
x = gcn(x, edge_index)

AttributeError: 'NoneType' object has no attribute 'device'

# Graph attention network

In [ ]:
class GATlayer(nn.Module):
    """graph attention layer implementation
    """
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.144)
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.144)
        
        self.leakyrelu = nn.LeakyReLU(self.alpha)
        
    def forwrd(self, inp, adj):
        h = torch.mm(inp, self.W)
        N = h.size()[0]
        
        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))
        
        zero_vec = -9e15*(torch.ones_like(e))
        attention = torch.where(adj>0, e, zero_vec)
        
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.matmul(attention, h)
        
        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

## pytorch implementation 

In [ ]:
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid

import torch_geometric.transforms as T

In [ ]:
name_data = 'Cora'
dataset = Planetoid(root= './' + name_data, name = name_data)
dataset.transform = T.NormalizeFeatures()

In [ ]:
class GAT(nn.Module):
    def __init__(self):
        super(GAT,self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        
        self.conv1 = GATConv(dataset.num_features, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, dataset.num_classes, heads=self.out_head, dropout=0.6)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index 
        
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = GAT().to(device)

data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

model.train()
for epoch in range(2000):
    model.train()
    optimizer.zero_grad()
    
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    
    if epoch % 100 == 0:
        print(loss)
        
    loss.backward()
    optimizer.step()

In [2]:
## graph convolutions
class GCNModel(torch.nn.Module):
    def __init__(self, output_dim, 
                 hidden_dim, 
                 node_features_dim,
                 edge_features_dim=None):
        super(GCNModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.conv1 = GraphConv(node_features_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, hidden_dim)
        self.conv3 = GraphConv(hidden_dim, hidden_dim)
        self.conv4 = GraphConv(hidden_dim, hidden_dim)
        self.conv5 = GraphConv(hidden_dim, hidden_dim)
        self.conv6 = GraphConv(hidden_dim, hidden_dim)
        
        self.fc1 = Linear(hidden_dim, hidden_dim)
        self.fc2 = Linear(hidden_dim, output_dim)
        
    def forward(self, x, node_index, batch):
        x = F.relu(self.conv1(x, node_index))
        x = F.relu(self.conv2(x, node_index))
        x = F.relu(self.conv3(x, node_index))
        x = F.relu(self.conv4(x, node_index))
        x = F.relu(self.conv5(x, node_index))
        x = F.relu(self.conv6(x, node_index))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return x

In [ ]:
def node_features(graph_data):
    properies = ['one_hot_atom', 'node_dm', 'totalNumHs', 
                 'totalatomDegree', 'hybridization', 'MO_contribution',
                 'atomic_charges', 'bonded_valence']
    properties_to_concat = []
    for prop in properies:
        if len(graph_data[prop].shape) == 1:
            properties_to_concat.append(graph_data[prop].reshape(-1, 1))
        else:
            properties_to_concat.append(graph_data[prop])
    graph_data.node_features = torch.cat(properties_to_concat, 1)
    return graph_data

## Training

In [ ]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=True)
val_loader = DataLoader(val_data, batch_size=10, shuffle=True)

In [ ]:
train_loader

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device:', device)

In [ ]:
model = GCNModel(output_dim=345*2, 
                 hidden_dim=2000, node_features_dim=1009).to(device) #, 

optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)
#scheduler 
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                          mode='min', 
                                                          patience=5,
                                                          threshold=1e-6
                                                         )
lr_scheduler2 =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=5000, gamma=0.1)

loss = torch.nn.MSELoss()                            

In [ ]:
def training_GCN(train_loader, val_loader, model, optimizer, loss_func, N_epochs):
    for epoch in tqdm(range(N_epochs)):
        for batch_train in tqdm(train_loader, leave=True, total=len(train_loader)):
            batch_train.to(device)
            #batch_val.to(device)
            
            out_train = model(batch_train.node_features, batch_train.edge_index, batch_train.batch, node_features_dim=batch_train.node_features.shape[1])
            #print(type(out_train), out_train.shape)
            loss_train = loss_func(out_train.flatten(), torch.tensor(batch_train.y).flatten())
            
            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()
            
            
        lr_scheduler2.step()
        print(optimizer.param_groups[0]['lr'])
            #break
                
        print(f'Epoch: {epoch}, Loss train : {loss_train}')

In [ ]:
training_GCN(train_loader, val_loader, model, optimizer, loss, N_epochs=10000)

In [ ]:
with torch.no_grad():
                out_val = model(batch_val.node_dm, batch_val.edge_index, batch_val.batch)
                loss_val = loss_func(out_val.flatten(), torch.tensor(batch_val.y).flatten())
            
, Loss valid : {loss_val}

# 8 GAT layers + linear layers as transformer 

In [ ]:
# convert pairwise distance to higher order features using RBF
def rbf_smearing(distances, sigma):
    """function to return smeared distance
    """
    return torch.exp(- distances / (2* sigma**2))

loading the dataset

In [5]:
gnn_layers = {
    'GCN' : geom_nn.GCNConv,
    'GAT' : geom_nn.GATConv,
    'Graphconv' : geom_nn.GraphConv,
    'GATv2' : geom_nn.GATv2Conv
             }

In [34]:
class GraphData(nn.Module):
    """
    class to get graph data from atromic coordinates and atom types
    """
    def __init__(self, n_neighbors, radius):
        super().__init__()
        self.n_neighbors = n_neighbors
        self.radius = radius
        #self.transform = transforms.Compose([transforms.ToTensor()])
        
    def _atomType_embedding(self, elements):
        if isinstance(elements, list)\
        and len(elements) != 0:
            pass
        else:
            raise ValueError("the elements should be a non empty list")
        i = len(elements)
        one_hot_m = np.eye(i, i)
        np.random.shuffle(one_hot_m)
        return {k:v for k, v in zip(elements, one_hot_m)}
    
    def pairwise_dists(self, atomic_coords):
        nbrs = NearestNeighbors(
            n_neighbors=self.n_neighbors, 
            radius=self.radius,
            algorithm='ball_tree'
        ).fit(atomic_coords)
        distances, inds = nbrs.kneighbors(atomic_coords)
        return distances, inds
    
    def _make_graph(self, atomic_coords, elements, atomic_numbers):
        G = nx.DiGraph()
        dists, inds = self.pairwise_dists(atomic_coords)
        
        all_edges = []
        for i, (atom_dists, atom_neighbors) in enumerate(zip(dists, inds)):
            for atom_dist, atom_neighbor \
            in zip(atom_dists[1:], atom_neighbors[1:]):
                all_edges.append((i, atom_neighbor, {'r_ij' : atom_dist}))
                
        G.add_edges_from(all_edges)
        
        all_nodes = []
        all_one_hot = self._atomType_embedding(list(set(elements)))
        G.add_nodes_from([(node, {'atom_type' : all_one_hot[atom], 'z' : z})
                         for node, (atom, z) in enumerate(zip(elements, atomic_numbers))
                         ])
        return G
    def forward(self, atomic_coords, elements, atomic_numbers):
        G = self._make_graph(atomic_coords, elements, atomic_numbers)
        return from_networkx(G)

test the GraphData class

In [ ]:
test_graph = GraphData(n_neighbors=30, radius=10)

atomic_coords, _, energy, _, elements = ggggh_dataset[1000]

test_g = test_graph(atomic_coords, elements)

#nx.draw(test_g, with_labels=True)
test_g

test RBFExpansion

In [ ]:
r_ij = test_g.r_ij

rbf_test = RBFExpansion(r_max=1, gap=0.2)
rbf_r_ij = rbf_test(r_ij, minmax=True)

rbf_r_ij.view(49, -1).shape

test the class of RBFExpansion

In [ ]:
rbfexp = RBFExpansion(r_max=1, gap=0.05)

length_scaler = StandardScaler()

scaled_rij = length_scaler.fit_transform(r_ij.reshape(-1, 1))

minmax = MinMaxScaler()
minmax_rij = minmax.fit_transform(scaled_rij)

rbf_rij = rbfexp(torch.tensor(minmax_rij).float())

## making the residual block for Linear layers

In [6]:
class ResidualBlockLinear(nn.Module):
    def __init__(self, c_in, c_out):
        super(ResidualBlockLinear, self).__init__()
        self.residual = nn.Sequential(
            nn.BatchNorm1d(c_in),
            nn.ReLU(),
            nn.Linear(c_in, c_out),
            nn.BatchNorm1d(c_out),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(c_out, c_out),
        )
        
    def _init_parameters(self):
        for layer in self.residual:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                
    def forward(self, x):
        residual = x
        out = self.residual(x)
        out += residual
        return out

## making the residual block for GNN layers

In [7]:
class ResidualBlockGAT(nn.Module):
    def __init__(self, c_in, c_out, heads, layer_name='GATv2', dropout=0.1, **kwargs):
        super(ResidualBlockGAT, self).__init__()
        if layer_name == 'GATv2':
            gnn_layer = gnn_layers[layer_name]
            self.residual = [
                nn.BatchNorm1d(c_in),
                nn.ReLU(),
                nn.Dropout(),
                gnn_layer(
                    c_in,
                    c_out,
                    heads=heads,
                    dropout=dropout
                ),
                nn.BatchNorm1d(c_out*heads),
                nn.ReLU(),
                nn.Dropout(),
                gnn_layer(
                    c_out*heads,
                    c_out,
                    heads=heads,
                    dropout=dropout
                ),
            ]
            self.layers = nn.ModuleList(self.residual)
        else:
            pass
        
    def forward(self, x, edge_index):
        residual = x
        for layer in self.layers:
            if isinstance(layer, geom_nn.MessagePassing): 
                out = layer(x, edge_index) 
            else:
                out = layer(x)
        out += residual
        return out                            

# ResNets

In [8]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks : list, c_in, c_hidd, c_out):
        super(ResNet, self).__init__()
        self.c_in = c_in
        self.c_hidd = c_hidd
        
        res_blocks = []
        self.in_layer = nn.Linear(c_in, c_hidd)
        for size_block in num_blocks:
            res_blocks.append(self._make_block(block, size_block))
            
        self.blocks = nn.ModuleList(res_blocks)
        self.out_layer = nn.Linear(c_hidd, c_out)
        
    def _make_block(self, block, num_layers):
        layers = []
        self.c_in = self.c_hidd
        for num_layer in range(num_layers):
            layers.append(block(self.c_in, self.c_hidd))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.in_layer(x)
        for layer in self.blocks:
            out = layer(out)
        out = self.out_layer(out)
        return out

In [9]:
class GATResNet(nn.Module):
    def __init__(self, 
                 block, 
                 num_blocks : list, 
                 c_in, c_hidd, 
                 c_out, heads, 
                 **kwargs):
        super(GATResNet, self).__init__()
        self.c_in=c_in
        self.c_hidd=c_hidd
        self.heads=heads
        
        #first layer of the GATResNet
        gnn_layer = gnn_layers['GATv2']
        self.layer_in=gnn_layer(c_in, c_hidd, heads=heads)
            
            
        res_blocks=[]
        for size_block in num_blocks:
            res_blocks.append(self._make_layer(block, size_block))
        self.res_blocks=nn.ModuleList(*res_blocks)
        
        #out layer of the GATResNet
        self.layer_out=gnn_layer(c_hidd*heads, c_out, heads=1)
        
        
    def _make_layer(self, block, num_layers):
        layers=[]
        for num_layer in range(num_layers):
            layers.append(block(
                self.c_hidd*self.heads,
                self.c_hidd,
                heads=self.heads
            ))
        return nn.ModuleList(layers)#nn.Sequential(*layers)
    
    def forward(self, x, edge_index):
        out = self.layer_in(x, edge_index)
        for layer in self.res_blocks:
            #for layer in block:
            out = layer(out, edge_index) 
        out = self.layer_out(out, edge_index)
        return out

In [7]:
class InitialEmbedding(nn.Module):
    def __init__(self, c_in, c_out, init_method='kaiming'):
        super().__init__()
        self.fc = nn.Linear(c_in, c_out)
        self.embedding = nn.Embedding(11, 3)
        self.init_method = init_method
        
    def _init_parameters(self):
            for layer in [*self.fc, self.embedding]:
                if self.init_method == 'xavier':
                    if isinstance(layer, nn.Linear):
                        nn.init.xavier_uniform_(layer.weight)
                elif self.init_method == 'kaiming':
                    if isinstance(layer, nn.Linear):
                        nn.init.kaiming_uniform_(layer.weight, mode='fan_in')
                else:
                    raise NotImplementedError
        
    def forward(self, x, rbf_ij, z):
        z = self.embedding(torch.tensor(z))
        x = torch.cat((x, rbf_ij, z), 1)
        out = F.relu(self.fc(x.float()))
        return out

test linear transformation on the dataset

In [169]:
class GNNModel(nn.Module):
    
    def __init__(self, 
                 in_channels, 
                 hidden_channels, 
                 out_channels,
                 in_heads=1,
                 out_heads=1,
                 num_layers=4, 
                 layer_name='GAT', 
                 dp_rate=0.1, 
                 **kwargs):
        super().__init__()
        self.dp_rate = dp_rate
        self.hid = hidden_channels
        self.in_head = in_heads
        self.out_head = out_heads
        #c_hidden = self.hid
        gnn_layer = gnn_layers[layer_name]
        all_layers = []
        
        for i in range(num_layers - 1):
            all_layers += [
                gnn_layer(
                in_channels, self.hid,
                heads=self.in_head, 
                dropout=self.dp_rate),
                nn.ReLU(),
                nn.Dropout(),
                ]
            in_channels = self.hid*self.in_head
            
        
        all_layers += [
            gnn_layer(
                self.hid*self.in_head, 
                out_channels,
                heads=self.out_head)
                      ]
        
        self.layers = nn.ModuleList(all_layers)
        
        
    def forward(self, x, edge_index):
        for layer in self.layers:
            if isinstance(layer, geom_nn.MessagePassing):
                x = layer(x, edge_index)
            else:
                x = layer(x)
        return x

In [170]:
class GNNModel(nn.Module):
    
    def __init__(self, 
                 gnn_layer : str,
                 in_channels, 
                 hidden_channels, 
                 out_channels,
                 in_heads=1,
                 out_heads=1,
                 num_layers=4, 
                 dp_rate=0.1, 
                 **kwargs):
        super().__init__()

        self.dp_rate = dp_rate
        self.in_head = in_heads
        
        
        all_layers = []
        for i in range(num_layers - 1):
            all_layers += [
                self._type_layer(
                    gnn_layer,
                    in_channels,
                    hidden_channels,
                    heads=in_heads
                ),
                nn.ReLU(),
                nn.Dropout(),
            ]
            if gnn_layer == 'GCN':
                in_channels = hidden_channels
            else:
                in_channels = hidden_channels*self.in_head 
        
        if gnn_layer == 'GCN':
            all_layers += [
                self._type_layer(
                    gnn_layer,
                    hidden_channels, 
                    out_channels,
                    heads=None
                )
            ]
        else:
            all_layers += [
                self._type_layer(
                    gnn_layer,
                    hidden_channels*self.in_head, 
                    out_channels,
                    heads=out_heads
                )
            ]

        self.layers = nn.ModuleList(all_layers)
        
    def _type_layer(self, type_gnn : str, c_in, c_out, **kwargs):
        
        if type_gnn == 'GCN':
            gnn_layer = geom_nn.GCNConv(
                c_in,
                c_out
            )
        elif type_gnn == 'GAT':
            gnn_layer = geom_nn.GATConv(
                c_in,
                c_out,
                heads=heads,
                dropout=self.dp_rate,
            )
        elif type_gnn == 'GATv2':
            gnn_layer = geom_nn.GATv2Conv(
                c_in,
                c_out,
                heads=heads,
                dropout=self.dp_rate
            )
        else:
            raise NotImplementedError
        
        return gnn_layer

    def forward(self, x, edge_index):
        for layer in self.layers:
            if isinstance(layer, geom_nn.MessagePassing):
                x = layer(x, edge_index)
            else:
                x = layer(x)
        return x

In [219]:
class GNNModel(nn.Module):
    
    def __init__(self,
                 in_channels, 
                 hidden_channels, 
                 out_channels,
                 in_heads=1,
                 num_layers=4, 
                 dp_rate=0.1, 
                 **kwargs):
        super().__init__()
    
        
        all_layers = []
        for i in range(num_layers - 1):
            all_layers += [
                geom_nn.GATv2Conv(
                    in_channels,
                    hidden_channels,
                    heads=in_heads
                ),
                nn.ReLU(),
                #nn.Dropout(),
            ]
            in_channels = hidden_channels*in_heads 
            
        all_layers += [
                geom_nn.GATv2Conv(
                    hidden_channels*in_heads,
                    out_channels,
                ),
            ResidualBlockLinear(
            out_channels,
            out_channels
            ),
            ResidualBlockLinear(
            out_channels,
            out_channels
            )
            #nn.ReLU(),
            #nn.Dropout()
        ]

        self.layers = nn.ModuleList(all_layers)

    def forward(self, x, edge_index):
        for layer in self.layers:
            if isinstance(layer, geom_nn.MessagePassing):
                x = layer(x, edge_index)
            else:
                x = layer(x)
        return x

In [220]:
class MDGAT(nn.Module):
    def __init__(self,
                 c_in_embedding,
                 c_out_embedding,
                 c_hidd_gat,
                 c_out_gat,
                 heads,
                 num_gnn_layers : int = 5,
                 **kwargs):
        super().__init__()
        
        #computing the pdists for QM9 dataset
        self.pwise_dists = GraphData(n_neighbors=10, radius=10)
        
        #initial_embedding
        self.initial_embedding = InitialEmbedding(
            c_in_embedding,
            c_out_embedding,
            init_method='kaiming',
        )
        
        
        self.GATModel = GNNModel(
                in_channels=c_out_embedding,
                hidden_channels=c_hidd_gat,
                out_channels=c_out_gat,
                in_heads=heads,
                num_layers=num_gnn_layers,
            )
        
        self.fc1 = nn.Linear(c_out_gat, c_out_gat*2)
        self.out = nn.Linear(c_out_gat*2, 1)
        
    def forward(self, data):
        batch, x, edge_index, r_ij, z = (
         data.batch, data.atom_type, data.edge_index, data.r_ij, data.z)
        
        #batch, x, edge_index, pos, z = (
            #data.batch, data.x, data.edge_index, data.pos, data.z)
        
        #pairwise distances for qm9
        #r_ij, _ = self.pwise_dists.pairwise_dists(pos.cpu().numpy())
        
        #r_ij = torch.tensor(r_ij).to(device)
        
        x = self.initial_embedding(x, r_ij, z)
        
        x = self.GATModel(x, edge_index)
        
        x = geom_nn.global_add_pool(x, batch)
        
        x = F.relu(self.fc1(x))
        output = self.out(x)
        
        return output    

In [ ]:
'''
        graph_data = self.graphData(
            atomic_coords[0].float(),
            elements,
        )
        edge_attr = self.rbf(graph_data.r_ij).view(
            graph_data.num_nodes, -1)
        
        node_feature = torch.cat(
            (graph_data.atom_type, edge_attr), 1
        )
        
        node_feature = self.transformer(node_feature.float())
                
        #the layer for converting data to graphs
        self.graphData = GraphData(
            n_neighbors=30,
            radius=10,
        )
        
        #expansion of pairwise distances
        self.rbf = RBFExpansion(gap=0.2)
'''

test of MDGAT model

In [ ]:
MDGAT_model = MDGAT(c_in=149, c_hidd=500, c_out=200)

y_pred = MDGAT_model(atomic_coords, elements)

convert all atomic dataset to graphs with y values as energies

In [35]:
make_graph = GraphData(n_neighbors=30, radius=10)

peptide_graphData = []
for i in range(0, len(peptide_dataset)):
    peptide_graphData.append(
        make_graph(
            peptide_dataset[i]['trajectory'], 
            peptide_dataset[i]['elements'],
            peptide_dataset[i]['z'],
        )
    )

In [7]:
for graph, data in \
zip(peptide_graphData, peptide_dataset):
    graph.y = torch.mul(data['energy'], 27.2114)

In [8]:
# expand rij distances 
rbf = RBFExpansion(gap=0.4)

for graph in peptide_graphData:
    graph.r_ij = rbf(graph.r_ij, minmax=True, 
                     standardScaler=False).view(graph.num_nodes, -1)
    graph.x = torch.cat(
        (graph.r_ij, graph.atom_type),
        1,
    )

In [43]:
peptide_graphData[0].r_ij.reshape(55, -1)

tensor([[1.0074, 1.0096, 1.4212,  ..., 8.1021, 8.1226, 8.1542],
        [1.0074, 1.6730, 1.9544,  ..., 8.2365, 8.6094, 8.6259],
        [1.0096, 1.6730, 2.0490,  ..., 8.8289, 8.8962, 8.9478],
        ...,
        [1.0206, 1.6978, 1.9525,  ..., 5.1206, 5.1605, 5.2387],
        [1.0745, 1.7402, 2.0480,  ..., 5.8236, 5.9187, 5.9229],
        [1.1119, 1.7306, 2.0354,  ..., 5.9677, 6.4463, 6.5308]],
       dtype=torch.float64)

# training

In [9]:
def get_lr(optimizer):
    for param_gp in optimizer.param_groups:
        lr = param_gp['lr']
        print(f'learning rate :{lr}')

In [10]:
learning_rate = 1e-4
batch_size = 100
epochs = 200

In [21]:
len(peptide_graphData)

4108

In [11]:
train_data, test_data = torch.utils.data.random_split(
    peptide_graphData, 
    [3000, 1108], 
    generator=torch.Generator().manual_seed(1)
)

In [12]:
train_loader = geom_data.DataLoader(train_data, batch_size, shuffle=True)

test_loader = geom_data.DataLoader(test_data, batch_size)
batch = next(iter(train_loader))

In [13]:
train_data[0]

Data(edge_index=[2, 1595], atom_type=[55, 4], z=[55], r_ij=[55, 87], num_nodes=55, y=[1], x=[55, 91])

In [14]:
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

In [27]:
!export CUDA_VISIBLE_DEVICES=0,1,2,3

In [15]:
MDGAT_model = MDGAT(
    c_in_embedding=94,
    c_out_embedding=94,
    c_hidd_gat=256,
    c_out_gat=128,
    heads=5,
    num_gnn_layers=4
).to(device)

NameError: name 'MDGAT' is not defined

In [203]:
MDGAT_model

MDGAT(
  (pwise_dists): GraphData()
  (initial_embedding): InitialEmbedding(
    (fc): Linear(in_features=94, out_features=94, bias=True)
    (embedding): Embedding(11, 3)
  )
  (GATModel): GNNModel(
    (layers): ModuleList(
      (0): GATv2Conv(94, 256, heads=5)
      (1): GATv2Conv(1280, 256, heads=5)
      (2): GATv2Conv(1280, 256, heads=5)
      (3): GATv2Conv(1280, 128, heads=1)
      (4): ResidualBlockLinear(
        (residual): Sequential(
          (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU()
          (2): Linear(in_features=128, out_features=128, bias=True)
          (3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (4): ReLU()
          (5): Dropout(p=0.5, inplace=False)
          (6): Linear(in_features=128, out_features=128, bias=True)
        )
      )
      (5): ResidualBlockLinear(
        (residual): Sequential(
          (0): BatchNorm1d(128, eps=1e-05, moment

In [160]:
#MDGAT_model = nn.DataParallel(MDGAT_model).to(device)

In [ ]:
#loss_fn = nn.MSELoss().to(device)
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(MDGAT_model.parameters(), learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',
    patience=20,
    factor=0.85,
    min_lr=1e-7
)

In [ ]:
for total_epochs in range(epochs):
    epoch_loss = 0
    total_graphs = 0
    MDGAT_model.train()
    for batch in train_loader:
        batch.to(device)
        optimizer.zero_grad()
        output = MDGAT_model(batch)
        loss = loss_fn(
             output,batch.y.unsqueeze(1))
        loss.backward()
        epoch_loss += loss.item()
        total_graphs += batch.num_graphs
        optimizer.step()
        scheduler.step(epoch_loss)
    train_avg_loss = epoch_loss / total_graphs
    val_loss = 0
    total_graphs = 0
    MDGAT_model.eval()
    for batch in test_loader:
        batch.to(device)
        output = MDGAT_model(batch)
        loss = loss_fn(
             output,batch.y.unsqueeze(1))
        val_loss += loss.item()
        total_graphs += batch.num_graphs
    val_avg_loss = val_loss / total_graphs
    print(f"Epochs: {total_epochs} | "
           f"epoch avg. loss: {train_avg_loss:.2f} | "
           f"validation avg. loss: {val_avg_loss:.2f}")

# Testing the model on different datasets

In [ ]:
import torch_geometric.data as geom_data

tu_dataset = torch_geometric.datasets.TUDataset(root='./', name="MUTAG")

torch.manual_seed(42)
tu_dataset.shuffle()
train_dataset = tu_dataset[:150]
test_dataset = tu_dataset[150:]

graph_train_loader = geom_data.DataLoader(train_dataset, batch_size=64, shuffle=True)
graph_val_loader = geom_data.DataLoader(test_dataset, batch_size=64) # Additional loader if you want to change to a larger dataset
graph_test_loader = geom_data.DataLoader(test_dataset, batch_size=64)

batch = next(iter(graph_train_loader))

batch

train_dataset[0]

In [196]:
MDGAT_model = MDGAT(
    c_in_embedding=24,
    c_out_embedding=24,
    c_hidd_gat=256,
    c_out_gat=128,
    heads=5,
    num_gnn_layers=3
).to(device)

In [191]:
MDGAT_model

MDGAT(
  (pwise_dists): GraphData()
  (initial_embedding): InitialEmbedding(
    (fc): Linear(in_features=24, out_features=24, bias=True)
    (embedding): Embedding(11, 3)
  )
  (GATModel): GNNModel(
    (layers): ModuleList(
      (0): GATv2Conv(24, 256, heads=5)
      (1): ResidualBlockLinear(
        (residual): Sequential(
          (0): BatchNorm1d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU()
          (2): Linear(in_features=1280, out_features=1280, bias=True)
          (3): BatchNorm1d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (4): ReLU()
          (5): Dropout(p=0.5, inplace=False)
          (6): Linear(in_features=1280, out_features=1280, bias=True)
        )
      )
      (2): GATv2Conv(1280, 256, heads=5)
      (3): ResidualBlockLinear(
        (residual): Sequential(
          (0): BatchNorm1d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU()
      

In [197]:
#loss_fn = nn.MSELoss().to(device)
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(MDGAT_model.parameters(), 1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',
    patience=20,
    factor=0.85,
    min_lr=1e-7
)

In [198]:
for total_epochs in range(100):
    epoch_loss = 0
    total_graphs = 0
    MDGAT_model.train()
    for batch in qm9_train_loader:
        batch.to(device)
        optimizer.zero_grad()
        output = MDGAT_model(batch)
        loss = loss_fn(
             output,batch.y[:, 10].unsqueeze(1))
        loss.backward()
        epoch_loss += loss.item()
        total_graphs += batch.num_graphs
        optimizer.step()
        scheduler.step(epoch_loss)
    train_avg_loss = epoch_loss / total_graphs
    val_loss = 0
    total_graphs = 0
    MDGAT_model.eval()
    for batch in qm9_test_loader:
        batch.to(device)
        output = MDGAT_model(batch)
        loss = loss_fn(
             output,batch.y[:, 10].unsqueeze(1))
        val_loss += loss.item()
        total_graphs += batch.num_graphs
    val_avg_loss = val_loss / total_graphs
    print(f"Epochs: {total_epochs} | "
           f"epoch avg. loss: {train_avg_loss:.2f} | "
           f"validation avg. loss: {val_avg_loss:.2f}")

Epochs: 0 | epoch avg. loss: 14.39 | validation avg. loss: 13.19
Epochs: 1 | epoch avg. loss: 3.08 | validation avg. loss: 4.45
Epochs: 2 | epoch avg. loss: 2.89 | validation avg. loss: 4.36
Epochs: 3 | epoch avg. loss: 2.86 | validation avg. loss: 4.49
Epochs: 4 | epoch avg. loss: 2.81 | validation avg. loss: 4.24
Epochs: 5 | epoch avg. loss: 2.90 | validation avg. loss: 4.35
Epochs: 6 | epoch avg. loss: 2.86 | validation avg. loss: 4.35
Epochs: 7 | epoch avg. loss: 2.88 | validation avg. loss: 4.50
Epochs: 8 | epoch avg. loss: 2.84 | validation avg. loss: 4.28
Epochs: 9 | epoch avg. loss: 2.85 | validation avg. loss: 4.37
Epochs: 10 | epoch avg. loss: 2.85 | validation avg. loss: 4.86
Epochs: 11 | epoch avg. loss: 2.85 | validation avg. loss: 4.58
Epochs: 12 | epoch avg. loss: 2.85 | validation avg. loss: 4.24
Epochs: 13 | epoch avg. loss: 2.86 | validation avg. loss: 4.60
Epochs: 14 | epoch avg. loss: 2.85 | validation avg. loss: 4.65
Epochs: 15 | epoch avg. loss: 2.83 | validation 

KeyboardInterrupt: 

# Using Pytorch lightning 

In [10]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

In [11]:
class LitMDGNN(pl.LightningModule):
    def __init__(self, gnn_model):
        super().__init__()
        self.gnn_model = gnn_model
        self.save_hyperparameters()
                
    def training_step(self, batch, batch_idx):
        output = self.gnn_model(batch)
        loss = loss_fn(output, batch.y[:, 10].unsqueeze(1))
        return loss / batch.num_graphs
    
    def test_step(self, batch, batch_idx):
        output = self.gnn_model(batch)
        test_loss = loss_fn(output, batch.y[:, 10].unsqueeze(1))
        self.log("test_loss", test_loss / batch.num_graphs)
        
    def validation_step(self, batch, batch_idx):
        output = self.gnn_model(batch)
        test_loss = loss_fn(output, batch.y[:, 10].unsqueeze(1))
        self.log("val_loss", test_loss / batch.num_graphs)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(), 
            1e-3
        )
        return optimizer

In [12]:
MDGAT_model = MDGAT(
    c_in_embedding=94,
    c_out_embedding=94,
    c_hidd_gat=256,
    c_out_gat=128,
    heads=5,
    num_gnn_layers=4
)

NameError: name 'MDGAT' is not defined

In [214]:
train_set_size = int(len(train_data)*0.8)
val_set_size = len(train_data) - train_set_size

seed=torch.Generator().manual_seed(42)
train_data, val_data = random_split(train_data, 
                                    [train_set_size, val_set_size],
                                    seed)

train_loader = geom_data.DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = geom_data.DataLoader(val_data, batch_size=64)

## early stopping

In [217]:
early_stopping_callbacks = EarlyStopping(monitor='val_loss', mode='min')

In [218]:
gat_model = LitMDGNN(MDGAT_model)

trainer = pl.Trainer(
    gpus=1, 
    max_epochs=100, 
    #callbacks=[early_stopping_callbacks],
)

trainer.fit(gat_model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name      | Type  | Params
------------------------------------
0 | gnn_model | MDGAT | 7.2 M 
------------------------------------
7.2 M     Trainable params
0         Non-trainable params
7.2 M     Total params
28.988    Total estimated model params size (MB)


ValueError: dictionary update sequence element #0 has length 1; 2 is required

## debugging 

In [100]:
trainer = pl.Trainer(fast_dev_run=10, gpus=1)
trainer.fit(gat_model, qm9_train_loader, qm9_val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 10 batch(es). Logging and checkpointing is suppressed.
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type  | Params
------------------------------------
0 | gnn_model | MDGAT | 1.8 M 
------------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params
7.023     Total estimated model params size (MB)


Epoch 0:  50%|█████     | 10/20 [00:00<00:00, 27.23it/s, loss=7.06, v_num=]
Validation: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 20/20 [00:00<00:00, 33.10it/s, loss=7.06, v_num=]

`Trainer.fit` stopped: `max_steps=10` reached.


Epoch 0: 100%|██████████| 20/20 [00:00<00:00, 33.02it/s, loss=7.06, v_num=]


## Run a sanity check 

In [106]:
trainer = pl.Trainer(num_sanity_val_steps=2, gpus=1)
trainer.fit(gat_model, qm9_train_loader, qm9_val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type  | Params
------------------------------------
0 | gnn_model | MDGAT | 1.8 M 
------------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params
7.023     Total estimated model params size (MB)


Epoch 0:  80%|███████▉  | 250/313 [00:08<00:02, 28.04it/s, loss=1.37, v_num=1.21e+7]
Validation: 0it [00:00, ?it/s]
Epoch 1:  80%|███████▉  | 250/313 [00:09<00:02, 27.62it/s, loss=2.23, v_num=1.21e+7]
Validation: 0it [00:00, ?it/s]
Epoch 2:  80%|███████▉  | 250/313 [00:08<00:02, 28.05it/s, loss=1.99, v_num=1.21e+7]
Validation: 0it [00:00, ?it/s]
Epoch 3:  80%|███████▉  | 250/313 [00:08<00:02, 28.14it/s, loss=1.69, v_num=1.21e+7]
Validation: 0it [00:00, ?it/s]
Epoch 4:  80%|███████▉  | 250/313 [00:08<00:02, 28.07it/s, loss=1.16, v_num=1.21e+7]
Validation: 0it [00:00, ?it/s]
Epoch 5:  80%|███████▉  | 250/313 [00:09<00:02, 27.65it/s, loss=2.2, v_num=1.21e+7] 
Validation: 0it [00:00, ?it/s]
Epoch 6:  80%|███████▉  | 250/313 [00:08<00:02, 28.17it/s, loss=1.66, v_num=1.21e+7] 
Validation: 0it [00:00, ?it/s]
Epoch 7:  80%|███████▉  | 250/313 [00:08<00:02, 28.00it/s, loss=2.03, v_num=1.21e+7]
Validation: 0it [00:00, ?it/s]
Epoch 8:  80%|███████▉  | 250/313 [00:09<00:02, 27.56it/s, loss=1.96, v

## checkpoints 

In [93]:
ckpt_path = './lightning_logs/version_12100283/checkpoints/epoch=75-step=19000.ckpt'
checkpoint = torch.load(ckpt_path)

checkpoint.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])

# experiments

In [11]:
from torch_geometric.nn import NNConv, GATConv, GCNConv, GATv2Conv
import torch.nn.functional as F

In [12]:
class ExampleNet(torch.nn.Module):
    def __init__(self, num_node_features, num_edge_features):
        super().__init__()
        conv1_net = nn.Sequential(
            nn.Linear(num_edge_features, 32),
            nn.ReLU(),
            nn.Linear(32, num_node_features*32))
        conv2_net = nn.Sequential(
            nn.Linear(num_edge_features, 32),
            nn.ReLU(),
            nn.Linear(32, 32*16))
        self.conv1 = NNConv(num_node_features, 32, conv1_net)
        self.conv2 = NNConv(32,16, conv2_net)
        self.fc_1 = nn.Linear(16, 32)
        self.out = nn.Linear(32, 1)
    def forward(self, data):
        batch, x, edge_index, edge_attr = (
            data.batch, data.x, data.edge_index, data.edge_attr)
        # First graph conv layer
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        # Second graph conv layer
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        x = global_add_pool(x,batch)
        x = F.relu(self.fc_1(x))
        output = self.out(x)
        return output

In [13]:
class ExampleNet(torch.nn.Module):
    def __init__(self, num_node_features, num_edge_features, heads=3):
        super().__init__()
        
        #self.fc1 = nn.Linear(num_node_features, 11)
        self.conv1 = GATv2Conv(num_node_features, 32, heads=heads)
        self.conv2 = GATv2Conv(32*heads, 20, heads=heads)
        self.conv3 = GATv2Conv(20*heads, 10, heads=heads)
        self.conv4 = GATv2Conv(10*heads, 5, heads=heads)
        #self.conv5 = GATConv(20*heads, 1, heads=heads)
        self.fc2 = nn.Linear(5*heads, 30)
        #self.fc_2 = nn.Linear(56, 10)
        self.out = nn.Linear(30, 1)
    def forward(self, data):
        batch, x, edge_index, edge_attr = (
            data.batch, data.x, data.edge_index, data.edge_attr)
        
        #x = F.relu(self.fc1(x))
        # First graph conv layer
        x = F.relu(self.conv1(x, edge_index))
        # Second graph conv layer
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = F.relu(self.conv4(x, edge_index))
        
        x = global_add_pool(x,batch)
        x = F.relu(self.fc2(x))
        #x = F.relu(self.fc_2(x))
        output = self.out(x)
        return output

In [14]:
qm9_node_feats, qm9_edge_feats = 11, 4

net = ExampleNet(qm9_node_feats, qm9_edge_feats).to(device)
#optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

NameError: name 'device' is not defined

In [191]:
optimizer = torch.optim.Adam(net.parameters(), 1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',
    patience=50,
    factor=0.85,
    min_lr=1e-7
)